In [1]:
from datetime import date

from create_data import recreate_databases, create_tables
from jobs.order_created import do
from utils import get_spark_session

In [2]:
spark = get_spark_session()

2021-11-25 12:14:48.107 | INFO     | utils:get_spark_session:7 - Creating Spark Session


In [3]:
recreate_databases(spark)

2021-11-25 12:15:05.259 | INFO     | create_data:recreate_databases:28 - Removing spark warehouse (path = 'spark-warehouse')
2021-11-25 12:15:05.321 | INFO     | create_data:recreate_databases:32 - Creating database 'standardized_glovo_live'
2021-11-25 12:15:10.785 | INFO     | create_data:recreate_databases:32 - Creating database 'mpcustomer_custom_events'
2021-11-25 12:15:10.891 | INFO     | create_data:recreate_databases:32 - Creating database 'mpcustomer_screen_views'
2021-11-25 12:15:10.982 | INFO     | create_data:recreate_databases:32 - Creating database 'enriched_custom_events'
2021-11-25 12:15:11.054 | INFO     | create_data:recreate_databases:32 - Creating database 'enriched_screen_views'


In [4]:
create_tables(spark)

2021-11-25 12:15:11.160 | INFO     | create_data:create_table:63 - Creating table 'standardized_glovo_live.cities'
2021-11-25 12:15:25.504 | INFO     | create_data:create_table:63 - Creating table 'standardized_glovo_live.devices'
2021-11-25 12:15:35.307 | INFO     | create_data:create_table:63 - Creating table 'mpcustomer_custom_events.order_created'


In [5]:
do(spark, exec_date=date(2021, 11, 19), n_days=3)

2021-11-25 12:15:46.732 | INFO     | loaders:load:30 - Loading 'mpcustomer_custom_events.order_created'
2021-11-25 12:15:47.240 | INFO     | loaders:load:30 - Loading 'standardized_glovo_live.cities'
2021-11-25 12:15:47.495 | INFO     | transformations.interfaces:_apply:13 - Applying transformation 'AddTimezone'
2021-11-25 12:15:47.566 | INFO     | writers:write:23 - Writting 'enriched_custom_events.order_created'


# Show input

In [6]:
spark.table("mpcustomer_custom_events.order_created").show()

+-----------------------+---------------+
|custom_attributes__city|p_creation_date|
+-----------------------+---------------+
|                    BCN|     2021-11-19|
|                    BCN|     2021-11-18|
|                    CAG|     2021-11-17|
|                    CAG|     2021-11-01|
+-----------------------+---------------+



# Show outputs

In [7]:
spark.table("enriched_custom_events.order_created").show()

+-----------------------+---------------+-------------+------------+
|custom_attributes__city|p_creation_date|    time_zone|country_code|
+-----------------------+---------------+-------------+------------+
|                    BCN|     2021-11-19|Europe/Madrid|          ES|
|                    BCN|     2021-11-18|Europe/Madrid|          ES|
|                    CAG|     2021-11-17|  Europe/Rome|          IT|
+-----------------------+---------------+-------------+------------+

